In [1]:
import pandas as pd
import random
import os
import time

# Input

In [2]:
bid_length = 16              # [ISP] Bid block length
max_load = 20000             # [MW] Maximum load that can be transported to the network
forecast_perc_av = 0.5       # Share of forecast that is based on the average EV behaviour, the rest is based on the behaviour of the previous week
EV_compensation = 0.75       # Share of the income that is used as compensation for EV-owners
fine_factor = 10             # This factor is 10 or 20, depending on whether the non-availability has been communicated to TenneT within 12 hours after the moment of delivery. Considering the fact that strategic behaviour is excluded from this research and that BSPs are obliged to report to TenneT if they are not able to fulfill their bid, it is assumed that aggregators will always report to TenneT within 12 hours and therefor the factor will be 10. 

# Charging properties
average_charging_speed = 11  # [kW] Average charging speed
flex_part = 0.1              # part of charging speed that is provided as flexibility

# Share of EVs that charge at different locations instead of parking without connecting the car to the loading station
charge_home = 0.9
charge_work = 0.5 
charge_work_visit = 0.2
charge_bringing = 0
charge_education = 0
charge_shop = 0
charge_visit = 0.05
charge_wandering = 0
charge_hobby = 0.05
charge_leisure = 0.05
charge_PC = 0
charge_other = 0
charge_list = [charge_home, charge_work, charge_work_visit, charge_bringing, charge_education, charge_shop, charge_visit, charge_wandering, charge_hobby, charge_leisure, charge_PC, charge_other]

In [3]:
col = ['Yearday', 'Weekday', 'Workday', 'Time', 'Home', 'Work', 'Work_Visit', 'Bringing', 'Education', 'Shop', 'Visit', 'Wandering', 'Hobby', 'Leisure', 'Personal_Care', 'Other', 'Driving_to_Home', 'Driving_to_Work', 'Driving_to_Work_Visit', 'Driving_to_Bringing', 'Driving_to_Education', 'Driving_to_Shop', 'Driving_to_Visit', 'Driving_to_Wandering', 'Driving_to_Hobby', 'Driving_to_Leisure', 'Driving_to_Personal_Care', 'Driving_to_Other']

# Import data

#### Import and prepare average mobility

In [4]:
EV_average = pd.read_csv('EV all years yearday.csv')
EV_average = EV_average[col]

# Calculate available MW for forecasting in the model
EV_average['Available MW'] = 0
counter = 0
for c in EV_average.columns[list(EV_average.columns).index('Home'):list(EV_average.columns).index('Other')+1]:
    EV_average['Available MW'] = EV_average['Available MW'] + (EV_average[c] * charge_list[counter] * average_charging_speed * flex_part / 1000)
    counter += 1

#### Import FCR- and load data

In [5]:
## Import FCR-price data
FCR_prices = pd.read_csv('FCR Settlement Price.csv').drop('Unnamed: 0', axis = 1)

## Import Load data
average_load = pd.read_csv('Loaddata average.csv').drop('Unnamed: 0', axis = 1)

# Deterministic model

In [6]:
special_days = [1, 125, 359, 360]

In [7]:
df = EV_average

df['Load'] = average_load['Load [MW]']
df['Residual Load'] = max_load - df['Load']            # Residual load that is available for charging of EVs
df['Residual Flex'] = flex_part * df['Residual Load']  # Max available capacity for flex, given the residual load on the network
df['Residual Flex'] = df[['Residual Load', 'Residual Flex']].min(axis=1)  # To make sure never more FCR is bid than the residual load on the electricity network
df['FCR Price'] = FCR_prices['Price']
df['Forecast'] = 0
df['Bid Size'] = 0
    
## Forecast & submit bids
df['Forecast'] = df['Available MW']

    
## Include load limitations
df['Forecast incl load'] = df[['Forecast', 'Residual Flex']].min(axis=1)
df['Limitation'] = 'EV'
df.loc[df['Forecast incl load'] != df['Forecast'], 'Limitation'] = 'Network'
    
for t in range(672, df.shape[0], bid_length):
    df.loc[t:t+(bid_length-1), 'Bid Size'] = max(0, min(list(df.loc[t:t+(bid_length-1), 'Forecast incl load'])))
    
## Settlement and KPI's
df['Fine'] = (df['Bid Size'] - df['Available MW'])
df.loc[df.Fine < 0, 'Fine'] = 0
df['Fine'] = df['Fine'] * fine_factor * df['FCR Price'] / 96
df['Income'] = df['Bid Size'] * df['FCR Price'] / 96
df['Costs'] = EV_compensation * df['Income']
df['Profit'] = df['Income'] - df['Costs'] - df['Fine']
    
# Save kpi's to df_output
df = df[672:]
df_output = pd.DataFrame()
df_output.loc[0, 'Average Available MW'] = sum(df['Available MW']) / len(df['Available MW'])
df_output.loc[0, 'Average Bid [MW]'] = sum(df['Bid Size']) / len(df['Bid Size'])
df_output.loc[0, 'Total Income [1000 €]'] = sum(df['Income']) / 1000
df_output.loc[0, 'Total Costs [1000 €]'] = sum(df['Costs']) / 1000
df_output.loc[0, 'Total Fines [1000 €]'] = sum(df['Fine']) / 1000
df_fines = df[df['Fine'] != 0]
df_output.loc[0, 'Number of Fines [#]'] = df_fines.shape[0]
df_output.loc[0, 'Total Profit [1000 €]'] = sum(df['Profit']) / 1000
df_output.loc[0, 'Number of ISPs with grid limitations [#]'] = len(df[df['Limitation'] == 'Network'])

In [8]:
df_output.to_csv('Output deterministic model.csv')

In [9]:
df_output = df_output[['Average Bid [MW]', 'Number of Fines [#]', 'Total Profit [1000 €]', 'Number of ISPs with grid limitations [#]']]
df_output['Average Bid [MW]'] = df_output['Average Bid [MW]'].round(2)
df_output['Total Profit [1000 €]'] = df_output['Total Profit [1000 €]'].round()

In [10]:
df_output.astype(object)

,Average Bid [MW],Number of Fines [#],Total Profit [1000 €],Number of ISPs with grid limitations [#]
0,35.73,0,1095,0
